In [50]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [51]:
#Substitua o link da sua busca aqui
completeUrl = 'https://bdtd.ibict.br/vufind/Search/Results?lookfor=atendimento+ubs&type=AllFields&filter%5B%5D=format%3A%22masterThesis%22'

In [52]:
maxQuantity = 25

splitDelimiter = '/vufind'
completeUrl = completeUrl.split(splitDelimiter, 1)

baseUrl = completeUrl[0]
query = splitDelimiter + completeUrl[1]

pageNumber = 1
researchCounter = 0

In [53]:
def getHTML(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    return soup

def getRowsInfos(soup, remainingQuantity):
    researchQuantityInPage = 0

    authors = soup.select('.author a')
    years = soup.find_all('div', {'id': 'datePublish'})
    titles = soup.find_all('a', {'class': 'title'}, limit=remainingQuantity)
    links = soup.find_all('a', {'class': 'fulltext'})

    rows = []

    for (author, year, title, link) in zip(authors, years, titles, links):
        author = author.text
        year = year.text.split()[-1]
        title = title.text.strip()
        link = link.get('href')

        rows.append({
          "Autor": [author],
          "Ano de Defesa": [year], 
          "Título": [title], 
          "Link": [link]
        })

        researchQuantityInPage += 1

    return [researchQuantityInPage, rows]

def addNewRowToDf(rows, dfWithoutNewRow):
    dfWithNewRow = pd.DataFrame(rows)

    dfWithNewRow = pd.concat([dfWithoutNewRow, dfWithNewRow])

    print(dfWithNewRow)

    return dfWithNewRow

In [54]:
url = baseUrl + query
soup = getHTML(url)

In [55]:
df = pd.DataFrame()

In [59]:
while researchCounter < maxQuantity:
  remaining = maxQuantity - researchCounter
  quantityAdded = 0
  if pageNumber == 1:
    print("Página:", pageNumber)

    [quantityAdded, rows] = getRowsInfos(soup, remaining)

    df = addNewRowToDf(rows, df)

    print("Trabalhos coletados:", quantityAdded)
  elif remaining > 0:
    print("Página:", pageNumber)

    try:
      nextQueryLink = soup.select('ul.pagination > li.active + li > a')[0].get('href')
      url = baseUrl + nextQueryLink
      soup = getHTML(url)
    except:
      print(f"Parece que a sua busca não retorna {maxQuantity} resultados")
      break

    [quantityAdded, rows] = getRowsInfos(soup, remaining)

    df = addNewRowToDf(rows, df)

    print("Trabalhos coletados:", quantityAdded)

  pageNumber = pageNumber + 1
  researchCounter += quantityAdded

In [60]:
print(df)

                                         Autor Ano de Defesa  \
0           [Sarmento, Maria das Graças Silva]        [2019]   
1      [Gurgel, Camila Augusta de Rubim Costa]        [2014]   
2                [Horta, Heloisa Helena Lemos]        [2007]   
3                [Horta, Heloisa Helena Lemos]        [2007]   
4                        [Marco Antonio Kulik]        [2015]   
5                  [Silvana Aparecida Sanavio]        [2016]   
6                [Carolina Pinheiro Perracini]        [2009]   
7                       [Fernandes, Alexandre]        [2013]   
8        [Medeiros, Deisyanne Camara Alves de]        [2012]   
9                [LISBOA, Livia Anniele Sousa]        [2015]   
10                        [Neiva, Tiago Sousa]        [2016]   
11    [Cavalcante, Cristiane das Gra??as Dias]        [2016]   
12                [Avelar, Carine de Oliveira]        [2017]   
13            [Araújo, Manuela Luchesi Brazil]        [2010]   
14  [Claro, Juliane Marcela Guimarães [U

In [58]:
print(f"Foram coletados {researchCounter} trabalhos. Seu arquivo está pronto!")

Foram coletados 25 trabalhos. Seu arquivo está pronto!
